In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd ..

/home/iovcharenko/Documents/NotWork/UCU/liner-algebra/ucu-linear-algebra-final-project


In [3]:
import numpy as np
import numpy.linalg as la
import pandas as pd
import matplotlib.pyplot as plt 

from pathlib import Path
from tqdm.cli import tqdm 
from scipy.sparse import csr_matrix
from surprise import SVD
from surprise import Dataset
from surprise import Reader

from src.als import AlsSVD
from src.metrics import rmse

In [4]:
data_folder = Path("data")

In [5]:
df = pd.read_csv(data_folder / "subsets" / "low-sparsity" / "records.csv")
# df = pd.read_csv(data_folder / "subsets" / "mid-sparsity" / "records.csv")
# df = pd.read_csv(data_folder / "subsets" / "high-sparsity" / "records.csv")

tr_df = df[df.split == "train"]
val_df = df[df.split == "val"]

## Check custom ALS-SVD

In [6]:
model = AlsSVD(k=50, n_epochs=40, reg=0.1)
model.fit(tr_df)

train loop: 100%|██████████| 40/40 [01:06<00:00,  1.67s/it]


In [7]:
val_preds = model.predict(val_df)
print("val rmse:", rmse(val_df.rating.values, val_preds))

predict loop: 28952it [00:02, 13657.52it/s]

val rmse: 0.8864294678998716


In [8]:
tr_preds = model.predict(tr_df)
print("train rmse:", rmse(tr_df.rating.values, tr_preds))

predict loop: 115805it [00:08, 13629.55it/s]

train rmse: 0.6286052170508475


## Check Dummy predictor (mean value)

In [9]:
rmse(val_df.rating.values, np.repeat(val_df.rating.mean(), len(val_df)))

1.0485102138998519

## Check SVD from Surprise lib 

In [10]:
reader = Reader()
data = Dataset.load_from_df(tr_df[['customer_id', 'movie_id', 'rating']], reader)
trainset = data.build_full_trainset()

model = SVD()
model.fit(trainset)

def predict(surprise_model, df):
    preds = []
    for _, item in tqdm(df.iterrows()):
        preds.append(surprise_model.predict(item.customer_id, item.movie_id).est)
    return preds

rmse(val_df.rating.values, predict(model, val_df))

28952it [00:02, 11435.98it/s]


0.8947077918786165